# Multimodalna Analiza Zmienności Rytmu Serca

**Autorzy:** Bartłomiej Tempka, Juliusz Wasieleski 
**Data:** Czerwiec 2025

## Opis projektu
Projekt polega na wykonaniu multimodalnej analizy zmienności rytmu serca (HRV) z wykorzystaniem trzech datasetów:
1. **Mechanocardiograms** - EKG + akcelerometria (Dataset 1)
2. **Cardiac patients with valvular diseases** - EKG + akcelerometria + żyroskopia (Dataset 2)  
3. **CEBSDB** - EKG + oddychanie + sejsmokardiografia (Dataset 3)

## Cele:
- Implementacja detektorów tętna dla różnych modalności
- Analiza HRV w dziedzinie czasu, częstotliwości i nieliniowa
- Porównanie wyników między modalnościami

In [1]:
# Importy podstawowe
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks, butter, filtfilt
import os
import warnings
warnings.filterwarnings('ignore')

# Konfiguracja wykresów
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10



## 1. Eksploracja Dataset 1: Mechanocardiograms

Ten dataset zawiera sygnały EKG wraz z akcelerometrią (mechanokardiografia). Sprawdźmy strukturę danych.


In [2]:
def load_mechanocardiogram(filepath):
    """Wczytanie danych mechanokardiogramów z pliku txt"""
    data = {}
    signals = []
    
    with open(filepath, 'r') as file:
        lines = file.readlines()
    
    header_end = 0
    for i, line in enumerate(lines):
        if line.startswith('[DATA]'):
            header_end = i + 1
            break
        elif 'Sample rate' in line:
            data['fs'] = int(line.split('=')[1].strip().replace('Hz', ''))
        elif 'Measurement length' in line:
            data['duration'] = line.split(':')[1].strip()
        elif line.startswith('Signal'):
            signal_info = line.strip().split(': ')
            signal_name = signal_info[1].split(',')[0]
            signals.append(signal_name)
    
    # Wczytaj dane numeryczne
    if header_end > 0:
        numeric_data = []
        for line in lines[header_end:]:
            if line.strip():
                values = [float(x) for x in line.strip().split()]
                numeric_data.append(values)
        
        data['signals'] = np.array(numeric_data).T
        data['signal_names'] = signals
        data['time'] = np.arange(len(data['signals'][0])) / data['fs']
    
    return data

## 2. Eksploracja Dataset 2: Cardiac Patients

Ten dataset zawiera dane pacjentów z wadami zastawkowymi serca, rejestrowane za pomocą sensorów Shimmer.


In [3]:
def load_cardiac_patient_data(filepath):
    """Wczytanie danych pacjentów kardiologicznych z pliku CSV Shimmer"""
    
    # Wczytaj plik CSV, pomijając pierwszą linię z sep=,
    try:
        df = pd.read_csv(filepath, skiprows=1)
        
        # Wyodrębnij podstawowe informacje
        data = {}
        
        # POPRAWKA: Konwertuj timestamp na liczbę
        timestamp_raw = df.iloc[:, 0].values  # Pierwsza kolumna to timestamp
        data['timestamp'] = pd.to_numeric(timestamp_raw, errors='coerce')
        
        # Usuń wiersze z nieprawidłowymi timestampami (NaN)
        valid_mask = ~np.isnan(data['timestamp'])
        data['timestamp'] = data['timestamp'][valid_mask]
        
        # Identyfikuj kolumny według nazw
        ecg_cols = [col for col in df.columns if 'ECG' in col and 'CAL' in col]
        acc_cols = [col for col in df.columns if 'Accel' in col and 'CAL' in col]
        gyro_cols = [col for col in df.columns if 'Gyro' in col and 'CAL' in col]
        
        # POPRAWKA: Konwertuj wszystkie sygnały na liczby
        if ecg_cols:
            ecg_raw = df[ecg_cols].values[valid_mask]
            data['ecg'] = pd.DataFrame(ecg_raw).apply(pd.to_numeric, errors='coerce').values
            data['ecg_names'] = ecg_cols
        
        if acc_cols:
            acc_raw = df[acc_cols].values[valid_mask]
            data['accelerometer'] = pd.DataFrame(acc_raw).apply(pd.to_numeric, errors='coerce').values
            data['acc_names'] = acc_cols
            
        if gyro_cols:
            gyro_raw = df[gyro_cols].values[valid_mask]
            data['gyroscope'] = pd.DataFrame(gyro_raw).apply(pd.to_numeric, errors='coerce').values
            data['gyro_names'] = gyro_cols
        
        # Usuń wiersze z NaN w sygnałach (jeśli występują)
        if 'ecg' in data:
            ecg_valid = ~np.isnan(data['ecg']).any(axis=1)
            for key in ['timestamp', 'ecg', 'accelerometer', 'gyroscope']:
                if key in data and key != 'timestamp':
                    data[key] = data[key][ecg_valid]
            if 'timestamp' in data:
                data['timestamp'] = data['timestamp'][ecg_valid]
        
        # Oszacuj częstotliwość próbkowania
        if len(data['timestamp']) > 1:
            dt = np.median(np.diff(data['timestamp']))
            data['fs'] = 1.0 / (dt / 1000.0)  # Convert ms to Hz
        
        # Czas w sekundach od początku
        data['time'] = (data['timestamp'] - data['timestamp'][0]) / 1000.0
        
        print(f"Pomyślnie wczytano {len(data['timestamp'])} próbek")
        
        # Sprawdź typy danych
        if 'ecg' in data:
            print(f"Typ danych EKG: {data['ecg'].dtype}")
        if 'accelerometer' in data:
            print(f"Typ danych akcelerometru: {data['accelerometer'].dtype}")
        if 'gyroscope' in data:
            print(f"Typ danych żyroskopu: {data['gyroscope'].dtype}")
        
        return data
        
    except Exception as e:
        print(f"Błąd wczytywania: {e}")
        import traceback
        traceback.print_exc()
        return None


## 3. Implementacja Detektorów Tętna

Implementujemy detektory dla różnych modalności:
- **EKG**: Klasyczny detektor zespołów QRS
- **SCG** (Sejsmokardiografia): Detektor na podstawie akcelerometru
- **GCG** (Żyrokardiografia): Detektor na podstawie żyroskopu


In [4]:
class HeartbeatDetector:
    """Klasa bazowa dla detektorów tętna"""
    
    def __init__(self, fs):
        self.fs = fs
    
    def bandpass_filter(self, signal, low_freq, high_freq, order=4):
        """Filtr pasmowo-przepustowy"""
        nyquist = self.fs / 2
        low = low_freq / nyquist
        high = high_freq / nyquist
        b, a = butter(order, [low, high], btype='band')
        return filtfilt(b, a, signal)
    
    def detect_peaks(self, signal, **kwargs):
        """Bazowa metoda detekcji pików"""
        raise NotImplementedError

class ECGDetector(HeartbeatDetector):
    """Detektor dla sygnałów EKG - Pan-Tompkins algorithm"""
    
    def detect_peaks(self, ecg_signal, min_peak_height=None, min_distance=None):
        """Detekcja zespołów QRS w sygnale EKG"""
        
        # Parametry domyślne
        if min_distance is None:
            min_distance = int(0.6 * self.fs)  # Minimum 100 BPM
        
        # Krok 1: Filtracja pasmowa (5-15 Hz dla QRS)
        filtered = self.bandpass_filter(ecg_signal, 5, 15)
        
        # Krok 2: Różniczkowanie
        diff_signal = np.diff(filtered)
        
        # Krok 3: Podnoszenie do kwadratu
        squared = diff_signal ** 2
        
        # Krok 4: Średnia ruchoma
        window_size = int(0.15 * self.fs)  # 150ms window
        integrated = np.convolve(squared, np.ones(window_size)/window_size, mode='same')
        
        # Krok 5: Detekcja pików
        if min_peak_height is None:
            min_peak_height = 0.35 * np.max(integrated)
        
        peaks, _ = find_peaks(integrated, 
                            height=min_peak_height,
                            distance=min_distance)
        
        return peaks

class SCGDetector(HeartbeatDetector):
    """Detektor dla sejsmokardiogramów (akcelerometr)"""
    
    def detect_peaks(self, acc_signal, axis=2, min_peak_height=None, min_distance=None):
        """Detekcja uderzeń serca z sygnału akcelerometru"""
        
        if min_distance is None:
            min_distance = int(0.4 * self.fs)  # Minimum 150 BPM
        
        # Wybierz odpowiednią oś lub oblicz magnitude
        if acc_signal.ndim == 1:
            signal_to_process = acc_signal
        else:
            if axis is None:
                # Magnitude wszystkich osi
                signal_to_process = np.sqrt(np.sum(acc_signal**2, axis=1))
            else:
                signal_to_process = acc_signal[:, axis]
        
        # Filtracja dla sygnałów kardiologicznych (1-40 Hz)
        filtered = self.bandpass_filter(signal_to_process, 1, 40)
        
        # Wzmocnienie przez podniesienie do kwadratu
        enhanced = filtered ** 2
        
        # Wygładzenie
        window_size = int(0.1 * self.fs)  # 100ms window
        smoothed = np.convolve(enhanced, np.ones(window_size)/window_size, mode='same')
        
        # Detekcja pików
        if min_peak_height is None:
            min_peak_height = 0.3 * np.max(smoothed)
        
        peaks, _ = find_peaks(smoothed,
                            height=min_peak_height,
                            distance=min_distance)
        
        return peaks

class GCGDetector(HeartbeatDetector):
    """Detektor dla żyrokardiogramów (żyroskop)"""
    
    def detect_peaks(self, gyro_signal, axis=2, min_peak_height=None, min_distance=None):
        """Detekcja uderzeń serca z sygnału żyroskopu"""
        
        if min_distance is None:
            min_distance = int(0.4 * self.fs)  # Minimum 150 BPM
        
        # Wybierz odpowiednią oś lub oblicz magnitude
        if gyro_signal.ndim == 1:
            signal_to_process = gyro_signal
        else:
            if axis is None:
                # Magnitude wszystkich osi
                signal_to_process = np.sqrt(np.sum(gyro_signal**2, axis=1))
            else:
                signal_to_process = gyro_signal[:, axis]
        
        # Filtracja dla sygnałów kardiologicznych (1-20 Hz)
        filtered = self.bandpass_filter(signal_to_process, 1, 20)
        
        # Wartość bezwzględna różniczki (wykrywa szybkie zmiany)
        diff_signal = np.abs(np.diff(filtered))
        diff_signal = np.append(diff_signal, diff_signal[-1])  # Restore length
        
        # Wygładzenie
        window_size = int(0.08 * self.fs)  # 80ms window
        smoothed = np.convolve(diff_signal, np.ones(window_size)/window_size, mode='same')
        
        # Detekcja pików
        if min_peak_height is None:
            min_peak_height = 0.4 * np.max(smoothed)
        
        peaks, _ = find_peaks(smoothed,
                            height=min_peak_height,
                            distance=min_distance)
        
        return peaks


## 4. Analiza HRV (Heart Rate Variability)

Implementujemy analizę zmienności rytmu serca w trzech domenach:
- **Dziedzina czasu**: AVNN, SDNN, RMSSD, pNN50
- **Dziedzina częstotliwości**: VLF, LF, HF, LF/HF
- **Analiza nieliniowa**: wykres Poincaré (SD1, SD2)


In [5]:
class HRVAnalyzer:
    """Klasa do analizy zmienności rytmu serca"""
    
    def __init__(self, rr_intervals):
        """
        Args:
            rr_intervals: odstępy RR w milisekundach
        """
        self.rr_intervals = np.array(rr_intervals)
        self.nn_intervals = self.filter_normal_intervals()  # Filtrowane odstępy NN
    
    def filter_normal_intervals(self, min_rr=300, max_rr=2000):
        """Filtracja nietypowych odstępów RR"""
        mask = (self.rr_intervals >= min_rr) & (self.rr_intervals <= max_rr)
        return self.rr_intervals[mask]
    
    def time_domain_analysis(self):
        """Analiza w dziedzinie czasu"""
        if len(self.nn_intervals) < 2:
            return {}
        
        # Podstawowe statystyki
        avnn = np.mean(self.nn_intervals)  # Średni odstęp NN
        sdnn = np.std(self.nn_intervals, ddof=1)  # Odchylenie standardowe
        
        # RMSSD - Root Mean Square of Successive Differences
        diff_nn = np.diff(self.nn_intervals)
        rmssd = np.sqrt(np.mean(diff_nn**2))
        
        # pNN50 - procent różnic > 50ms
        pnn50 = (np.sum(np.abs(diff_nn) > 50) / len(diff_nn)) * 100
        
        # pNN20 - procent różnic > 20ms (dodatkowy wskaźnik)
        pnn20 = (np.sum(np.abs(diff_nn) > 20) / len(diff_nn)) * 100
        
        return {
            'AVNN': avnn,
            'SDNN': sdnn,
            'RMSSD': rmssd,
            'pNN50': pnn50,
            'pNN20': pnn20,
            'HR_mean': 60000 / avnn if avnn > 0 else 0  # Średnie tętno
        }
    
    def frequency_domain_analysis(self, fs_resample=4):
        """Analiza w dziedzinie częstotliwości"""
        if len(self.nn_intervals) < 10:
            return {}
        
        # Interpolacja do równomiernie próbkowanego sygnału
        time_original = np.cumsum(self.nn_intervals) / 1000.0  # Convert to seconds
        time_new = np.arange(0, time_original[-1], 1/fs_resample)
        
        # Interpolacja liniowa
        rr_interpolated = np.interp(time_new, time_original, self.nn_intervals)
        
        # Detrending
        rr_detrended = rr_interpolated - np.mean(rr_interpolated)
        
        # FFT
        fft_vals = np.fft.fft(rr_detrended)
        fft_freqs = np.fft.fftfreq(len(rr_detrended), 1/fs_resample)
        
        # Power spectral density (PSD)
        psd = np.abs(fft_vals)**2
        
        # Definicje pasm częstotliwości (Hz)
        vlf_band = (0.0033, 0.04)
        lf_band = (0.04, 0.15)
        hf_band = (0.15, 0.4)
        
        # Znajdź indeksy dla każdego pasma
        def get_band_power(freq_range):
            mask = (fft_freqs >= freq_range[0]) & (fft_freqs <= freq_range[1])
            return np.sum(psd[mask]) if np.any(mask) else 0
        
        vlf_power = get_band_power(vlf_band)
        lf_power = get_band_power(lf_band)
        hf_power = get_band_power(hf_band)
        
        total_power = vlf_power + lf_power + hf_power
        
        # Znormalizowane moce
        lf_norm = (lf_power / (lf_power + hf_power)) * 100 if (lf_power + hf_power) > 0 else 0
        hf_norm = (hf_power / (lf_power + hf_power)) * 100 if (lf_power + hf_power) > 0 else 0
        
        # Stosunek LF/HF
        lf_hf_ratio = lf_power / hf_power if hf_power > 0 else 0
        
        return {
            'VLF_power': vlf_power,
            'LF_power': lf_power,
            'HF_power': hf_power,
            'Total_power': total_power,
            'LF_norm': lf_norm,
            'HF_norm': hf_norm,
            'LF_HF_ratio': lf_hf_ratio
        }
    
    def nonlinear_analysis(self):
        """Analiza nieliniowa - wykres Poincaré"""
        if len(self.nn_intervals) < 2:
            return {}
        
        # Współrzędne wykresu Poincaré
        rr_n = self.nn_intervals[:-1]
        rr_n1 = self.nn_intervals[1:]
        
        # SD1 - krótkookresowa zmienność (szerokość chmury punktów)
        diff_rr = rr_n1 - rr_n
        sd1 = np.std(diff_rr, ddof=1) / np.sqrt(2)
        
        # SD2 - długookresowa zmienność (długość chmury punktów)
        sum_rr = rr_n1 + rr_n
        sd2 = np.std(sum_rr, ddof=1) / np.sqrt(2)
        
        # SD1/SD2 ratio
        sd_ratio = sd1 / sd2 if sd2 > 0 else 0
        
        return {
            'SD1': sd1,
            'SD2': sd2,
            'SD1_SD2_ratio': sd_ratio,
            'poincare_rr_n': rr_n,
            'poincare_rr_n1': rr_n1
        }
    
    def comprehensive_analysis(self):
        """Kompletna analiza HRV"""
        results = {}
        results.update(self.time_domain_analysis())
        results.update(self.frequency_domain_analysis())
        results.update(self.nonlinear_analysis())
        return results

def peaks_to_rr_intervals(peaks, fs):
    """Konwersja pozycji pików na odstępy RR w milisekundach"""
    if len(peaks) < 2:
        return np.array([])
    
    rr_samples = np.diff(peaks)
    rr_ms = (rr_samples / fs) * 1000  # Convert to milliseconds
    return rr_ms



## 5. Przykład Analizy Multimodalnej

Teraz przeprowadzimy przykładową analizę na rzeczywistych danych z obu datasetów.


In [6]:
def multimodal_analysis_example():
    """Przykład analizy multimodalnej dla jednego pacjenta"""
    
    try:
        # Załaduj dane pacjenta
        patient_data = load_cardiac_patient_data('Raw_Recordings/CP-01-Raw.csv')
        
        if patient_data is None:
            print("Nie udało się załadować danych pacjenta")
            return
        
        fs = patient_data['fs']
        duration_minutes = patient_data['time'][-1] / 60
        
        print(f"=== Analiza Pacjenta CP-01 ===")
        print(f"Czas nagrania: {duration_minutes:.1f} minut")
        print(f"Częstotliwość próbkowania: {fs:.1f} Hz")
        print()
        
        # Wybierz fragment danych (np. pierwsze 5 minut)
        max_samples = int(5 * 60 * fs)  # 5 minut
        end_idx = min(max_samples, len(patient_data['time']))
        
        # === ANALIZA EKG ===
        if 'ecg' in patient_data:
            print("--- Analiza EKG ---")
            # Użyj pierwszego odprowadzenia EKG
            ecg_signal = patient_data['ecg'][:end_idx, 0]
            
            # Detekcja QRS
            ecg_detector = ECGDetector(fs)
            qrs_peaks = ecg_detector.detect_peaks(ecg_signal)
            
            if len(qrs_peaks) > 1:
                # Analiza HRV
                rr_intervals = peaks_to_rr_intervals(qrs_peaks, fs)
                hrv_analyzer = HRVAnalyzer(rr_intervals)
                hrv_results_ecg = hrv_analyzer.comprehensive_analysis()
                
                print(f"Wykryto {len(qrs_peaks)} zespołów QRS")
                print(f"Średnie tętno: {hrv_results_ecg.get('HR_mean', 0):.1f} BPM")
                print(f"SDNN: {hrv_results_ecg.get('SDNN', 0):.1f} ms")
                print(f"RMSSD: {hrv_results_ecg.get('RMSSD', 0):.1f} ms")
                print(f"LF/HF: {hrv_results_ecg.get('LF_HF_ratio', 0):.2f}")
            else:
                print("Nie wykryto wystarczającej liczby zespołów QRS")
                hrv_results_ecg = {}

        # === ANALIZA AKCELEROMETRU ===
        if 'accelerometer' in patient_data:
            print("\n--- Analiza Akcelerometru ---")
            # Użyj osi Z akcelerometru
            acc_signal = patient_data['accelerometer'][:end_idx, 2]  # Z-axis
            
            # Detekcja uderzeń serca
            scg_detector = SCGDetector(fs)
            scg_peaks = scg_detector.detect_peaks(acc_signal, axis=None)  # Single axis
            
            if len(scg_peaks) > 1:
                # Analiza HRV
                rr_intervals_scg = peaks_to_rr_intervals(scg_peaks, fs)
                hrv_analyzer_scg = HRVAnalyzer(rr_intervals_scg)
                hrv_results_scg = hrv_analyzer_scg.comprehensive_analysis()
                
                print(f"Wykryto {len(scg_peaks)} uderzeń serca (SCG)")
                print(f"Średnie tętno: {hrv_results_scg.get('HR_mean', 0):.1f} BPM")
                print(f"SDNN: {hrv_results_scg.get('SDNN', 0):.1f} ms")
                print(f"RMSSD: {hrv_results_scg.get('RMSSD', 0):.1f} ms")
                print(f"LF/HF: {hrv_results_scg.get('LF_HF_ratio', 0):.2f}")
            else:
                print("Nie wykryto wystarczającej liczby uderzeń serca (SCG)")
                hrv_results_scg = {}
        
        # === ANALIZA ŻYROSKOPU ===
        if 'gyroscope' in patient_data:
            print("\n--- Analiza Żyroskopu ---")
            # Użyj osi Z żyroskopu
            gyro_signal = patient_data['gyroscope'][:end_idx, 2]  # Z-axis
            
            # Detekcja uderzeń serca
            gcg_detector = GCGDetector(fs)
            gcg_peaks = gcg_detector.detect_peaks(gyro_signal, axis=None)  # Single axis
            
            if len(gcg_peaks) > 1:
                # Analiza HRV
                rr_intervals_gcg = peaks_to_rr_intervals(gcg_peaks, fs)
                hrv_analyzer_gcg = HRVAnalyzer(rr_intervals_gcg)
                hrv_results_gcg = hrv_analyzer_gcg.comprehensive_analysis()
                
                print(f"Wykryto {len(gcg_peaks)} uderzeń serca (GCG)")
                print(f"Średnie tętno: {hrv_results_gcg.get('HR_mean', 0):.1f} BPM")
                print(f"SDNN: {hrv_results_gcg.get('SDNN', 0):.1f} ms")
                print(f"RMSSD: {hrv_results_gcg.get('RMSSD', 0):.1f} ms")
                print(f"LF/HF: {hrv_results_gcg.get('LF_HF_ratio', 0):.2f}")
            else:
                print("Nie wykryto wystarczającej liczby uderzeń serca (GCG)")
                hrv_results_gcg = {}
        
        print("\n=== Analiza zakończona ===")
        
    except Exception as e:
        print(f"Błąd podczas analizy: {e}")
        import traceback
        traceback.print_exc()


multimodal_analysis_example()


Pomyślnie wczytano 124838 próbek
Typ danych EKG: float64
Typ danych akcelerometru: float64
Typ danych żyroskopu: float64
=== Analiza Pacjenta CP-01 ===
Czas nagrania: 8.1 minut
Częstotliwość próbkowania: 256.0 Hz

--- Analiza EKG ---
Wykryto 10 zespołów QRS
Średnie tętno: 0.0 BPM
SDNN: 0.0 ms
RMSSD: 0.0 ms
LF/HF: 0.00

--- Analiza Akcelerometru ---
Wykryto 5 uderzeń serca (SCG)
Średnie tętno: 70.1 BPM
SDNN: 116.0 ms
RMSSD: 164.1 ms
LF/HF: 0.00

--- Analiza Żyroskopu ---
Nie wykryto wystarczającej liczby uderzeń serca (GCG)

=== Analiza zakończona ===


## 6. Plan Dalszych Prac

### Następne kroki dla projektu:

1. **Testowanie detektorów** na większej liczbie pacjentów
2. **Porównanie dokładności** różnych modalności
3. **Analiza korelacji** między wynikami z różnych sygnałów
4. **Grupowanie pacjentów** (zdrowi vs. z wadami zastawkowymi)
5. **Walidacja** z referencyjnymi metodami
6. **Wizualizacja wyników** - wykresy porównawcze
7. **Analiza statystyczna** różnic między grupami

### Możliwe rozszerzenia:
- Implementacja algorytmów uczenia maszynowego dla klasyfikacji
- Analiza w czasie rzeczywistym
- Optymalizacja parametrów detektorów
- Fusion różnych modalności dla lepszej dokładności
